In [1]:
%pip install transformers sentence-transformers langchain faiss-cpu langchain-community

In [2]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [3]:
# Initialize Sentence Transformer embeddings
model_name = "all-MiniLM-L6-v2" 
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipykernel_13366/1985587213.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/home/work/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-07 12:43:29.118145: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tu

In [4]:
df = pd.read_csv("./dataset/combined_dataset.csv")
df.head(2)

,title,sub_info,subject,rating,level,institution,about,what_you_will_learn,syllabus,course_url,skills_you_will_gain
0,HarvardX: The Architectural Imagination,Learn fundamental principles of architecture —...,Architecture,NaN,Introductory,HarvardX,Architecture engages a culture’s deepest socia...,"How to read, analyze, and understand different...",Part I: Form and History Part II: The Technolo...,https://www.edx.org/learn/architecture/harvard...,NaN
1,MITx: Sustainable Building Design,"Learn and explore key scientific principles, t...",Architecture,NaN,Intermediate,MITx,"Meeting growing global energy demand, while mi...",Understand the scientific principles underlyin...,Week 1 - Energy Use in Buildings Week 2 - Unde...,https://www.edx.org/learn/sustainable-developm...,NaN


In [5]:
# Function to prepare data for embedding
def prepare_data_for_embedding(row):
    combined_info = f"""
    Title: {row['title']}
    Subject: {row['subject']}
    Description: {row['sub_info'] if pd.notna(row['sub_info']) else 'no data'}
    Level: {row['level'] if pd.notna(row['level']) else 'no data'}
    Institution: {row['institution'] if pd.notna(row['institution']) else 'no data'}
    About: {row['about'] if pd.notna(row['about']) else 'no data'}
    What You Will Learn: {row['what_you_will_learn'] if pd.notna(row['what_you_will_learn']) else 'no data'}
    Syllabus: {row['syllabus'] if pd.notna(row['syllabus']) else 'no data'}
    Skills You Will Gain: {row['skills_you_will_gain'] if pd.notna(row['skills_you_will_gain']) else 'no data'}
    Rating: {row['rating'] if pd.notna(row['rating']) else 'no data'}
    Course URL: {row['course_url']}
    """
    
    return combined_info.strip()

In [6]:
# Prepare data for embedding
df['combined_info'] = df.apply(prepare_data_for_embedding, axis=1)

In [7]:
df['combined_info'][2]

'Title: TokyoTechX: Japanese Architecture and Structural Design\n    Subject: Architecture\n    Description: In this revised course, fundamental and modern approaches to Japanese structural design will be explained using historical overviews and Tokyo-Tech’s campus buildings as case studies. Learners will be able to interpret and apply seismic design concepts like energy-dissipating braced frames, spine frames, seismic retrofit, seismic isolation and seismic design of spatial structures. Drawing from the instructor’s past 20 years of experience in design, considerations to sustainability, practical complexities and their solutions will be presented using the campus buildings as design examples with immersive 360-degree interactive videos. Learners will also get an insight into the architect-engineer collaboration through interviews with architectural professors.\n    Level: Intermediate\n    Institution: TokyoTechX\n    About: In this revised course, in depth video lectures cover vario

In [8]:
# Create a vector store using FAISS
texts = df['combined_info'].tolist()

metadatas = df[['title', 'sub_info', 'rating', 'subject', 'level', 'institution', 'course_url']].to_dict('records')

# Create the vector store
vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

In [9]:
# Save the FAISS index
vectorstore.save_local("faiss_index")

print("FAISS index saved to faiss_index directory")

FAISS index saved to faiss_index directory
